In [ ]:
# !uv add ipykernel python-dotenv langchain openai rapidocr-onnxruntime python-dotenv langchain-community faiss-cpu langchain-google-genai

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Data Ingestion

In [2]:
from langchain_community.document_loaders import TextLoader

In [3]:
os.getcwd()

'c:\\Users\\keith\\implementations\\MachineLearning_AI\\llmops_vault\\notebook'

In [4]:
loader = TextLoader("../data/agentic_ai.txt", encoding= "utf-8")
documents = loader.load()

In [5]:
documents

[Document(metadata={'source': '../data/agentic_ai.txt'}, page_content='Agentic AI refers to artificial intelligence systems that can\nautonomously plan, make decisions, and take actions to achieve specific\ngoals with minimal human intervention. Unlike traditional AI models that\nsimply respond to prompts or perform single-step predictions, agentic\nsystems are designed to reason through multi-step problems, interact\nwith tools, access external data sources, and adapt their strategies\nbased on feedback. These systems operate more like digital agents,\ncapable of goal-driven behavior rather than isolated task execution.\n\nOne of the defining characteristics of agentic AI is its ability to\ncombine reasoning, memory, and tool usage. For example, an agent may\nbreak a complex task into smaller steps, retrieve relevant information,\nexecute code, query databases, or call APIs before synthesizing a final\nanswer. This makes agentic AI particularly powerful in workflows such as\nresearch 

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [8]:
text_chunks=text_splitter.split_documents(documents)
text_chunks

[Document(metadata={'source': '../data/agentic_ai.txt'}, page_content='Agentic AI refers to artificial intelligence systems that can\nautonomously plan, make decisions, and take actions to achieve specific'),
 Document(metadata={'source': '../data/agentic_ai.txt'}, page_content='goals with minimal human intervention. Unlike traditional AI models that\nsimply respond to prompts or perform single-step predictions, agentic'),
 Document(metadata={'source': '../data/agentic_ai.txt'}, page_content='systems are designed to reason through multi-step problems, interact\nwith tools, access external data sources, and adapt their strategies'),
 Document(metadata={'source': '../data/agentic_ai.txt'}, page_content='based on feedback. These systems operate more like digital agents,\ncapable of goal-driven behavior rather than isolated task execution.'),
 Document(metadata={'source': '../data/agentic_ai.txt'}, page_content='One of the defining characteristics of agentic AI is its ability to\ncombine r

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"  # Gemini embedding model
)
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [23]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [ ]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

Document 1:
Agentic AI refers to artificial intelligence systems that can
autonomously plan, make decisions, and take actions to achieve specific
--------------------------------------------------
Document 2:
goals with minimal human intervention. Unlike traditional AI models that
simply respond to prompts or perform single-step predictions, agentic
--------------------------------------------------
Document 3:
One of the defining characteristics of agentic AI is its ability to
combine reasoning, memory, and tool usage. For example, an agent may
--------------------------------------------------
Document 4:
defined boundaries and align with human intent is critical. With proper
governance and responsible deployment, agentic AI has the potential to
--------------------------------------------------


In [13]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [14]:
prompt=ChatPromptTemplate.from_template(template)

In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [16]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",   # Free + active model
    temperature=0.3
)

In [31]:
from langchain_core.runnables import RunnablePassthrough
# RAG Chain
rag_chain = (
    {
        "context": retriever,              # your FAISS retriever
        "question": RunnablePassthrough()  # passes user question
    }
    | prompt
    | llm_model
    | output_parser
)


In [32]:
response = rag_chain.invoke("What is Agentic AI?")
print(response)


Agentic AI refers to artificial intelligence systems capable of autonomously planning, making decisions, and taking actions to achieve specific goals. These systems operate with minimal human intervention. Unlike traditional AI models that primarily respond to prompts or perform single-step predictions, agentic AI exhibits a higher degree of independence. A defining characteristic is its ability to combine reasoning, memory, and tool usage. For instance, an agent might use various tools to accomplish a complex task. It is critical for agentic AI to operate within defined boundaries and align with human intent. With proper governance and responsible deployment, agentic AI holds significant potential.
